In [1]:
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
key = os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    api_key=key,
    temperature=0.6
)

name = llm.invoke('I want to open a restaurant for German food. Suggest a single fancy name for this, with no explanation')

In [5]:
name.content

'Hausmanns Keller'

In [6]:
from langchain_core.prompts import PromptTemplate

In [7]:
prompt_name = PromptTemplate.from_template(
    """ 
    I want to open a restaurant for {cuisine} food. Suggest a single fancy name for this,
      with no explanation and no string quotes
    """

)

prompt_menu = PromptTemplate.from_template(
    """ 
    Suggest some menu items for {restaurant_name}'. Return it as a comma seperated list. Not more than 5 items.
    I want only the json with the restaurant name as one key value pair and the menu items as the second
    NO PREAMBLE AND NO ```
    """

)

In [8]:
chain = prompt_menu | prompt_name | llm
res = chain.invoke({'cuisine','Chinese'})

In [39]:
first_chain = prompt_name | llm
second_chain = prompt_menu | llm
ans = second_chain.invoke(first_chain.invoke({'cuisine':'Chinese'})).content

In [40]:
ans

'{"Jade Imperial Palace": "Kung Pao Chicken, Beef with Broccoli, Shrimp Fried Rice, Egg Foo Young, Wonton Soup"}'

In [11]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [31]:
tools = load_tools(tool_names=['wikipedia'], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,handle_parsing_errors=True)

In [32]:
agent.invoke({
    'input':"When was Elon Musk born? What is his age right now in 2023? "
})



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find out Elon Musk's birthdate and calculate his age in 2023.

Action: wikipedia
Action Input: Elon Musk

Observation: Page: Elon Musk
Summary: Elon Reeve Musk  (; born June 28, 1971) is a businessman known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. His other involvements include ownership of X Corp., the company that operates the social media platform X (formerly Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. In November 2024, United States president-elect Donald Trump appointed Musk as the co-chair of the proposed Department of Government Efficiency (DOGE) in the second Trump administration. Musk is the wealthiest individual in the world; as of November 2024, Forbes estimates his net worth to be US$304 billion.
Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating 

{'input': 'When was Elon Musk born? What is his age right now in 2023? ',
 'output': 'Elon Musk was born on June 28, 1971. In 2023, he would be 52 years old.'}

In [16]:
load_dotenv()

os.environ['SERPAPI_API_KEY'] = os.getenv('SERP_API_KEY')

In [25]:
tools = load_tools(tool_names=['serpapi','llm-math'], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,handle_parsing_errors=True)

agent.run("What was the GDP of US in 2022 plus 5")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find the GDP of the US in 2022 and then add 5 to it.

Action: Search
Action Input: "US GDP 2022"
Observation: ["The U.S. is a country of 50 states covering a vast swath of North America, with Alaska in the northwest and Hawaii extending the nation’s presence into the Pacific Ocean. Major Atlantic Coast cities are New York, a global finance and culture center, and capital Washington, DC. Midwestern metropolis Chicago is known for influential architecture and on the west coast, Los Angeles' Hollywood is famed for filmmaking. ― Google", 'United States type: Country in North America.', 'United States entity_type: countries, locations, travel, competitions, military.', 'United States kgmid: /m/09c7w0.', 'Current-dollar GDP increased 9.2 percent, or $2.15 trillion, in 2022 to a level of $25.46 trillion, compared with an increase of 10.7 percent, ...', 'Current-dollar GDP increased 9.2 percent, or $2.15 trilli

'25446000000005'

In [34]:
from langchain.chains import LLMChain, SequentialChain

In [57]:
chain = LLMChain(llm=llm,prompt=prompt_name)
name = chain.invoke({'cuisine':'Indian'})

In [61]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_name, memory=memory)
chain.run('Mexican')
chain.run('Italian')
chain.run({'Indian'})

C:\Users\User\AppData\Local\Temp\ipykernel_12704\486847452.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [67]:
print(chain.memory.buffer)

Human: Mexican
AI: Fiesta del Sol
Human: ['Indian']
AI: Maharaja's Spice Bazaar
Human: Italian
AI: Bella Vita Ristorante


In [68]:
from langchain.chains import ConversationChain

In [72]:
convo = ConversationChain(llm=llm)
convo.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

In [73]:
convo.run('What team won the first rocket league championship series. Give only the name of the team')

'Dignitas'

In [74]:
convo.run("What is 5 + 5")

'10'

In [75]:
convo.run("Who were the players on the winning team")

'The players on the winning team Dignitas in the first Rocket League Championship Series were Kronovi, RSN, and Turbopolsa'

In [77]:
print(convo.memory.buffer)

Human: What team won the first rocket league championship series. Give only the name of the team
AI: Dignitas
Human: What is 5 + 5
AI: 10
Human: Who were the players on the winning team
AI: The players on the winning team Dignitas in the first Rocket League Championship Series were Kronovi, RSN, and Turbopolsa


In [82]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

convo = ConversationChain(llm=llm, memory=memory)

In [83]:
convo.run('What team won the first rocket league championship series. Give only the name of the team')

'Dignitas'

In [84]:
convo.run("What is 5 + 5")

'10'

In [85]:
convo.run("Who were the players on the winning team")

'The winning team, Dignitas, had the following players in the first Rocket League Championship Series: Kronovi, Rizzo, and Turbopolsa'